# Solving The Advection Equation

### Sometimes before you can make a hard thing easy you have to make an easy thing hard.

We're going to solve the Advection Equation here. 

$$
\partial_t u(t, x) + a \partial_x u(t, x) = 0
$$

Here $a$ is the advection velocity (dimensions Length / Time) and $u$ is the quantity being advected.  The exact solution is trivial to write down: $u(t, x) = u(x - at, 0)$, but that is not the point.  The advection equation is the simplest example of a _conservation law_ partial differential equation, and is used a great deal to test numerical algorithms.  Before you can use your algorithm for hydrodynamics, you have to show it can deal with advection.

Here we build a simple method to solve this equation to 1st order in space and time.  More sophisticated methods can go to substantially higher order.

### Spatial Grid

First we need to define our discretization of space.

This is a "cell-centered" grid that breaks up the interval $[a, b]$ into $N_x$ cells of width $\Delta x = (b-a)/N_x$.  There are $N_x+1$ _faces_ in between the cells: $x_{e,i} = a + i \Delta x$ and $N_x$ cell centers at $x_i = a + (i+1/2)\Delta x$.  The `make_grid` function also allows for $N_g$ _ghost zones_ on each side of the boundary, which are useful for applying boundary conditions.

In [3]:
function make_grid(a, b, Nx, Ng)
    
    # The desired cell-width
    dx = (b - a) / Nx
    
    # First get the edges of all the cells, adding ghost zones to the left and right
    xe = LinRange(a - Ng*dx, b + Ng*dx, Nx + 2*Ng + 1)
    
    # Now the cell centers are just the average of the left & right cell faces
    xL = xe[begin:end-1]
    xR = xe[begin+1:end]
    x = 0.5*(xL + xR)
    
    return dx, x, xe
end

make_grid (generic function with 1 method)

### Plotting Utilities

Just a couple useful functions for plotting our data.

## You may have to change GLMakie to WGLMakie 

In [2]:
using GLMakie
GLMakie.activate!()

function make_figure()
    fig = Figure()
    ax = Axis(fig[1,1])
    return fig, ax
end

function add_to_plot(ax, x, f)
    lines!(ax, x, f)
    # plot!(ax, x, f)
end

add_to_plot (generic function with 1 method)

### Initial Conditions

These functions will return an array that can be used as an initial state for our solve.

In [19]:
function init_bump(x, x0, width, amp)
    # A smooth bump of amplitude `amp` and width `w` at position `x0`.
    f = similar(x)
    for i in 1:length(x)
        if ((x[i]-x0)/width)^2 >= 1.0
            f[i] = 0.0
        else
            f[i] = amp*cos(0.5*pi*(x[i]-x0)/width)^2
        end
    end
    return f
end

function init_square(x, x0, width, amp)
    # A square wave of amplitude `amp` and width `w` at position `x0`.
    f = similar(x)
    for i in 1:length(x)
        if ((x[i]-x0)/width)^2 >= 1.0
            f[i] = 0.0
        else
            f[i] = amp
        end
    end
    return f
end

init_square (generic function with 1 method)

### Boundary Conditions

Every time step, we need to set the ghost zones to have appropriate values.  We will do this in two ways: a _periodic_ boundary condition that wraps the domain around on itself, and a _copy_ or _outflow_ boundary condition that copies the nearest real zone into the ghost zone (setting the gradient at the boundary to 0)

In [4]:
function no_BC(u)
    # do nothing
end

function periodic_L(u)
    # Periodic BC on the left edge.
    u[begin] = u[end-1]
end

function periodic_R(u)
    # Periodic BC on the right edge.
    u[end] = u[begin+1]
end

function copy_L(u)
    # Copy BC on the left edge.
    u[begin] = u[begin+1]
end

function copy_R(u)
    # Copy BC on the right edge.
    u[end] = u[end-1]
end

copy_R (generic function with 1 method)

## Grid Timestep Function

These functions update our grid using Forward-Euler.  The first performs a single step.  The second computes many steps until a desired time $t_final$.

In [16]:
function timestep(u, dx, dt, a, udot_f, bc_L_f, bc_R_f)
    N = length(u)
    
    udot = similar(u)
    
    # First compute the time derivative of u on for each (real) zone
    for i in 2:N-1 
        udot[i] = udot_f(u[i-1], u[i], u[i+1], a, dx)
    end

    # Now update each real zone in-place
    for i in 2:N-1 
        u[i] += dt * udot[i]
    end
    
    # Now apply the boundary condition functions to set the ghost zones.
    bc_L_f(u)
    bc_R_f(u)
end

function evolve(u, t_final, dx, dt, a, udot_f, bc_L_f, bc_R_f)
    t = 0
    
    while t < t_final

        if t + dt > t_final
            dt = t_final - t
        end
            
        timestep(u, dx, dt, a, udot_f, bc_L_f, bc_R_f)
        t += dt
    end
end

evolve (generic function with 1 method)

# Per-Cell update function -- where the magic happens!

This function gets called on each cell to compute the time-derivative: $\partial_t u = -a\partial_x u$

In [8]:
function udot_ftcs(uL, uC, uR, a, dx)
    # Write a function that computes du/dt for the advection equation.
end

udot_ftcs (generic function with 1 method)

### Test it out down here!